In [1]:
# import library
import pickle
import pandas as pd
import numpy as np

import xgboost as xgb
import lightgbm as lgb

from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="1", 
        allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
    )
)
set_session(tf.Session(config=config))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# read data
df = pd.read_csv('../data/feature_selection_positive.csv')


# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [4]:
# define and fit
clf = rf()
clf = clf.fit(X_train, y_train)

# result append to list
t = []
t.append(round(clf.score(X_test, y_test)*100, 2))
pickle.dump(clf, open('../model/rf_pos_fs.sav', 'wb'))

In [5]:
# define and fit
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)

pickle.dump(gbm, open('../model/lgbm_pos_fs.sav', 'wb'))
# result append to list
t.append(round(gbm.score(X_test, y_test)*100, 2))

[1]	valid_0's multi_logloss: 1.29182
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.21089
[3]	valid_0's multi_logloss: 1.14382
[4]	valid_0's multi_logloss: 1.08417
[5]	valid_0's multi_logloss: 1.03056
[6]	valid_0's multi_logloss: 0.985742
[7]	valid_0's multi_logloss: 0.941113
[8]	valid_0's multi_logloss: 0.904691
[9]	valid_0's multi_logloss: 0.870271
[10]	valid_0's multi_logloss: 0.840251
[11]	valid_0's multi_logloss: 0.812001
[12]	valid_0's multi_logloss: 0.790457
[13]	valid_0's multi_logloss: 0.767174
[14]	valid_0's multi_logloss: 0.745236
[15]	valid_0's multi_logloss: 0.725653
[16]	valid_0's multi_logloss: 0.708227
[17]	valid_0's multi_logloss: 0.689849
[18]	valid_0's multi_logloss: 0.673774
[19]	valid_0's multi_logloss: 0.657902
[20]	valid_0's multi_logloss: 0.642649
[21]	valid_0's multi_logloss: 0.630764
[22]	valid_0's multi_logloss: 0.618836
[23]	valid_0's multi_logloss: 0.607679
[24]	valid_0's multi_logloss: 0.598533
[25]	valid_0's mu

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [6]:
# set parameters
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [7]:
# define and fit
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# result append to list
pickle.dump(model, open('../model/xgb_pos_fs.sav', 'wb'))
t.append(round(model.score(X_test, y_test)*100, 2))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [8]:
tf.set_random_seed(42)

# make keras model

inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_pos_fs.h5')
t.append(round(model.evaluate(X_test, y_test_for_keras, verbose=0)[1]*100, 2))

Train on 1340 samples, validate on 336 samples
Epoch 1/100
1340/1340 [==============================] - 2s 1ms/step - loss: 1.8874 - acc: 0.2321 - val_loss: 1.4034 - val_acc: 0.2262
Epoch 2/100
1340/1340 [==============================] - 0s 67us/step - loss: 1.7915 - acc: 0.2634 - val_loss: 1.3867 - val_acc: 0.2798
Epoch 3/100
1340/1340 [==============================] - 0s 63us/step - loss: 1.7366 - acc: 0.2716 - val_loss: 1.4068 - val_acc: 0.2857
Epoch 4/100
1340/1340 [==============================] - 0s 65us/step - loss: 1.6335 - acc: 0.2925 - val_loss: 1.3852 - val_acc: 0.2976
Epoch 5/100
1340/1340 [==============================] - 0s 67us/step - loss: 1.5817 - acc: 0.3045 - val_loss: 1.3496 - val_acc: 0.3244
Epoch 6/100
1340/1340 [==============================] - 0s 60us/step - loss: 1.6045 - acc: 0.3022 - val_loss: 1.3249 - val_acc: 0.3333
Epoch 7/100
1340/1340 [==============================] - 0s 66us/step - loss: 1.5334 - acc: 0.3254 - val_loss: 1.3328 - val_acc: 0.3661
Ep

1340/1340 [==============================] - 0s 65us/step - loss: 0.6472 - acc: 0.7328 - val_loss: 0.8718 - val_acc: 0.6607
Epoch 61/100
1340/1340 [==============================] - 0s 60us/step - loss: 0.6485 - acc: 0.7231 - val_loss: 0.8785 - val_acc: 0.6667
Epoch 62/100
1340/1340 [==============================] - 0s 66us/step - loss: 0.6392 - acc: 0.7291 - val_loss: 0.8840 - val_acc: 0.6488
Epoch 63/100
1340/1340 [==============================] - 0s 60us/step - loss: 0.5931 - acc: 0.7649 - val_loss: 0.8852 - val_acc: 0.6429
Epoch 64/100
1340/1340 [==============================] - 0s 62us/step - loss: 0.6230 - acc: 0.7418 - val_loss: 0.8633 - val_acc: 0.6607
Epoch 65/100
1340/1340 [==============================] - 0s 61us/step - loss: 0.5926 - acc: 0.7746 - val_loss: 0.8467 - val_acc: 0.6577
Epoch 66/100
1340/1340 [==============================] - 0s 63us/step - loss: 0.5705 - acc: 0.7694 - val_loss: 0.8369 - val_acc: 0.6518
Epoch 67/100
1340/1340 [==============================

In [9]:
f = []
f.append(t)

In [10]:
f

[[72.32, 84.52, 83.04, 66.37]]

In [12]:
#read data
df = pd.read_csv('../data/feature_selection_negative.csv')

# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [13]:
# define and fit 
clf = rf()
clf = clf.fit(X_train, y_train)

# result append to list
t = []
t.append(round(clf.score(X_test, y_test)*100, 2))
pickle.dump(clf, open('../model/rf_ng_fs.sav', 'wb'))

In [14]:
# define and fit
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)

pickle.dump(gbm, open('../model/lgbm_ng_fs.sav', 'wb'))
# result append to list
t.append(round(gbm.score(X_test, y_test)*100, 2))

[1]	valid_0's multi_logloss: 1.30732
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.24253
[3]	valid_0's multi_logloss: 1.17804
[4]	valid_0's multi_logloss: 1.13119
[5]	valid_0's multi_logloss: 1.0872
[6]	valid_0's multi_logloss: 1.04603
[7]	valid_0's multi_logloss: 1.00813
[8]	valid_0's multi_logloss: 0.973739
[9]	valid_0's multi_logloss: 0.944839
[10]	valid_0's multi_logloss: 0.914158
[11]	valid_0's multi_logloss: 0.890913
[12]	valid_0's multi_logloss: 0.867582
[13]	valid_0's multi_logloss: 0.849518
[14]	valid_0's multi_logloss: 0.83157
[15]	valid_0's multi_logloss: 0.82026
[16]	valid_0's multi_logloss: 0.803277
[17]	valid_0's multi_logloss: 0.791577
[18]	valid_0's multi_logloss: 0.777358
[19]	valid_0's multi_logloss: 0.769046
[20]	valid_0's multi_logloss: 0.759218
[21]	valid_0's multi_logloss: 0.751213
[22]	valid_0's multi_logloss: 0.744431
[23]	valid_0's multi_logloss: 0.735025
[24]	valid_0's multi_logloss: 0.727463
[25]	valid_0's multi_l

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [15]:
# setting parameters
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [16]:
# define and fit 
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# result append to list
t.append(round(model.score(X_test, y_test)*100, 2))
pickle.dump(model, open('../model/xgb_ng_fs.sav', 'wb'))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [17]:
tf.set_random_seed(42)

# make keras model

inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_ng_fs.h5')
t.append(round(model.evaluate(X_test, y_test_for_keras, verbose=0)[1]*100, 2))

Train on 373 samples, validate on 94 samples
Epoch 1/100
373/373 [==============================] - 1s 3ms/step - loss: 2.2138 - acc: 0.2064 - val_loss: 1.3822 - val_acc: 0.3723
Epoch 2/100
373/373 [==============================] - 0s 43us/step - loss: 1.9182 - acc: 0.2761 - val_loss: 1.3613 - val_acc: 0.4681
Epoch 3/100
373/373 [==============================] - 0s 43us/step - loss: 1.8160 - acc: 0.3164 - val_loss: 1.4258 - val_acc: 0.5106
Epoch 4/100
373/373 [==============================] - 0s 44us/step - loss: 1.6460 - acc: 0.3887 - val_loss: 1.5049 - val_acc: 0.5000
Epoch 5/100
373/373 [==============================] - 0s 46us/step - loss: 1.6544 - acc: 0.3968 - val_loss: 1.5190 - val_acc: 0.5000
Epoch 6/100
373/373 [==============================] - 0s 42us/step - loss: 1.6293 - acc: 0.3914 - val_loss: 1.4847 - val_acc: 0.4894
Epoch 7/100
373/373 [==============================] - 0s 50us/step - loss: 1.5146 - acc: 0.3619 - val_loss: 1.4169 - val_acc: 0.5106
Epoch 8/100
373/37

Epoch 62/100
373/373 [==============================] - 0s 40us/step - loss: 0.6653 - acc: 0.6917 - val_loss: 0.8776 - val_acc: 0.6277
Epoch 63/100
373/373 [==============================] - 0s 40us/step - loss: 0.6872 - acc: 0.7078 - val_loss: 0.8776 - val_acc: 0.6383
Epoch 64/100
373/373 [==============================] - 0s 40us/step - loss: 0.6412 - acc: 0.7292 - val_loss: 0.8746 - val_acc: 0.6383
Epoch 65/100
373/373 [==============================] - 0s 41us/step - loss: 0.7212 - acc: 0.6810 - val_loss: 0.8740 - val_acc: 0.6383
Epoch 66/100
373/373 [==============================] - 0s 41us/step - loss: 0.6239 - acc: 0.7373 - val_loss: 0.8757 - val_acc: 0.6383
Epoch 67/100
373/373 [==============================] - 0s 43us/step - loss: 0.6434 - acc: 0.7212 - val_loss: 0.8756 - val_acc: 0.6383
Epoch 68/100
373/373 [==============================] - 0s 42us/step - loss: 0.6567 - acc: 0.7292 - val_loss: 0.8772 - val_acc: 0.6383
Epoch 69/100
373/373 [==============================] -

In [18]:
f.append(t)

In [21]:
# save to csv
pd.DataFrame(
    f, 
    columns=['Random Forest', 'LightGBM', 'XGBoost', 'Keras'],
    index=['Positive', 'Negative']
).to_csv('../result/feature_selection.csv')

In [ ]:
f